In [1]:
import os
import h5py
import matplotlib.pyplot as plt
import numpy as np
from mpl_toolkits.mplot3d import Axes3D
import tensorflow as tf
from tensorflow import keras
import gc
from sys import getsizeof
import datetime
import random

In [2]:
def load_data(res, source_labels, dataset='train', size=100, verbosity = False):
    data = np.zeros((0, res, res, res))
    labels = []
    for i, label in enumerate(source_labels):
        filename = './output/' + label + '/output-' + dataset + '-' + str(res) + '.h5'
        if verbosity:
            print(filename)
        file = h5py.File(filename, 'r')
        data = np.concatenate((data, file['tensor']))
        labels = np.concatenate((labels, np.full((len(file['tensor'])), i)))
        file.close()
        gc.collect()
    return (data, labels)


In [3]:
def prepare_model(res, num_classes, verbosity = 0):
    model = keras.Sequential([
        keras.layers.Flatten(input_shape=(res, res, res)),
        keras.layers.Dense(res*2, activation='relu'),
        keras.layers.Dense(res, activation='relu'),
        keras.layers.Dense(num_classes, activation='softmax')
    ])

    learning_rate = 0.0001

    model.compile(optimizer=keras.optimizers.Adam(lr=learning_rate),
                  loss='sparse_categorical_crossentropy',
                  metrics=['accuracy'])
    if (verbosity == 1):
        print(model.summary())
    return model


In [4]:
def train(model, training, training_labels, res, verbosity = 0):
    log_dir = "logs/fit/" + str(res) + '-' + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
    tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=1)

    validation_split = 0.1
    epochs = 30
    
    model.fit(training, 
              training_labels, 
              epochs=epochs,
              verbose=verbosity,
              #validation_split=validation_split,
              callbacks=[tensorboard_callback])

In [5]:
#base_labels = ['bathtub', 'bed', 'chair', 'desk', 'dresser', 'monitor', 'night_stand', 'sofa', 'table', 'toilet']
all_labels = ['airplane', 'bathtub', 'bed', 'bench', 'bookshelf', 'bottle', 'bowl', 'car', 'chair', 'cone', 
                 'cup', 'curtain', 'desk', 'door', 'dresser', 'flower_pot', 'glass_box', 'guitar','keyboard', 'lamp',
                 'laptop', 'mantel', 'monitor', 'night_stand', 'person', 'piano', 'plant', 
                 'radio', 'range_hood', 'sink', 'sofa', 'stairs', 'stool', 'table', 'tent', 'toilet', 'tv_stand', 
                 'vase', 'wardrobe', 'xbox']
#num_classes = len(base_labels)

In [6]:
subclases = [10, 20, 30, 40]
resolutions = [24, 32, 48]

for j, num_classes in enumerate(subclases):
    base_labels = all_labels[:num_classes]
    for i,res in enumerate(resolutions):
        training, training_labels = load_data(res, base_labels, 'train', 100)
        model = prepare_model(res, num_classes)
        train(model, training, training_labels, res)
        test, test_labels = load_data(res, base_labels, 'test', 100)
        test_loss, test_acc = model.evaluate(test,  test_labels, verbose=2)
        print('\nTest accuracy for ' + str(num_classes) + ' classes width res ' + str(res) + ':', test_acc)
        gc.collect()

23/23 - 0s - loss: 0.3989 - accuracy: 0.9113

Test accuracy for 10 classes width res 24: 0.9112675786018372
23/23 - 0s - loss: 0.3360 - accuracy: 0.9085

Test accuracy for 10 classes width res 32: 0.908450722694397
23/23 - 0s - loss: 0.5114 - accuracy: 0.9070

Test accuracy for 10 classes width res 48: 0.9070422649383545
38/38 - 0s - loss: 0.6279 - accuracy: 0.8527

Test accuracy for 20 classes width res 24: 0.8527454137802124
38/38 - 0s - loss: 0.6704 - accuracy: 0.8594

Test accuracy for 20 classes width res 32: 0.8594009876251221
38/38 - 0s - loss: 0.7966 - accuracy: 0.8511

Test accuracy for 20 classes width res 48: 0.8510815501213074
59/59 - 0s - loss: 1.0815 - accuracy: 0.7548

Test accuracy for 30 classes width res 24: 0.7548179626464844
59/59 - 0s - loss: 1.1833 - accuracy: 0.7741

Test accuracy for 30 classes width res 32: 0.7740899324417114
59/59 - 1s - loss: 1.2951 - accuracy: 0.7682

Test accuracy for 30 classes width res 48: 0.7682012915611267
78/78 - 0s - loss: 1.2599 - a

In [ ]:
def draw(img, expected = '', predicted = ''):
    fig = plt.figure(figsize=[10,10])
    ax = fig.gca(projection='3d')
    cube1 = (img[:,:,:] >= 1)
    ax.voxels(cube1, facecolors="blue")
    plt.title('Expected: {}\n Predicted: {}'.format(expected, predicted), y=-0.01)
    plt.show()

In [ ]:
if not len(test):
    test, test_labels = load_data(dim, base_labels, 'test', 100)
test_loss, test_acc = model.evaluate(test,  test_labels, verbose=2)
p = model.predict(test)
print('\nTest accuracy:', test_acc)

In [ ]:
index = random.randint(0, len(test))

predicted_index = np.argmax(p[index])
expected = base_labels[int(test_labels[index])]
predicted = base_labels[predicted_index]
draw(test[index], expected, predicted)
index